In [9]:
import pandas as pd
from Acesso import Login
from Query import Query
from Tempo import DataHora

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [10]:
querys={
    
    'TargetEstatico':
    
    """
    
    IF MONTH(GETDATE())=1

        SELECT * FROM netfeira.vw_targetestatico
        WHERE YEAR([Data de Faturamento])=YEAR(GETDATE())-1 AND MONTH([Data de Faturamento])=12

    ELSE

        SELECT * FROM netfeira.vw_targetestatico
        WHERE YEAR([Data de Faturamento])=YEAR(GETDATE()) AND MONTH([Data de Faturamento])=MONTH(GETDATE())-1    
    
    """,
    
    'Produto':
    
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """
    
}

In [11]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [13]:
tabelas_df['TargetEstatico']=tabelas_df['TargetEstatico'].loc[tabelas_df['TargetEstatico']['Tipo de Operação']!='OUTROS']

In [17]:
tabelas_df['TargetEstatico'].columns

Index(['Data de Emissão', 'Data de Faturamento', 'Pedido', 'Nfe', 'ID Empresa',
       'ID Cliente', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabelas', 'SKU', 'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA',
       'Total Geral', 'Total Venda', 'Comissão R$', 'Margem Bruta R$',
       'Cad Vendedor', 'Situação', 'Peso Bruto KG', 'Peso Líquido KG'],
      dtype='object')

In [18]:
tabelas_df['Produto'].columns

Index(['SKU', 'Cód. Fabricante', 'Produto', 'Status', 'Fabricante',
       'Departamento', 'Seção', 'Categoria', 'Linha', 'Grupo de Produtos',
       'Grupo Indústria', 'Grupo de Meta', 'Grupo MCCAIN', 'Linha Fabrica',
       'Grupo de Promotores', 'Peso Liquido', 'Peso Bruto', 'Fator CX',
       'Unid. CMP', 'Unid. STK', 'Fator CMP'],
      dtype='object')

In [23]:
consolidado_df=tabelas_df['TargetEstatico'][['SKU','Peso Líquido KG','Total Geral']].groupby(['SKU'],as_index=False).sum()

In [30]:
tabelas_df['Produto']=tabelas_df['Produto'].loc[tabelas_df['Produto']['Fabricante'].str.contains('MCCAIN')]

In [32]:
consolidado_df=consolidado_df.merge(tabelas_df['Produto'],on='SKU',how='inner')[['SKU','Produto','Peso Líquido KG','Total Geral']]

In [34]:
consolidado_df.rename(columns={'SKU':'Code','Produto':'Nome','Peso Líquido KG':'Total Kg (LIQUIDO)','Total Geral':'Total preço'},inplace=True)

In [36]:
consolidado_df

,Code,Nome,Total Kg (LIQUIDO),Total preço
0,594,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",19897.5000,245641.05
1,614,CONGELADO ANEL DE CEBOLA MCCAIN (4KG),92.0000,4681.65
2,787,"CONGELADO BATATA MCCAIN ONE FRY (2,5KG)",0.0000,-2.00
3,847,"CONGELADO BATATA MCCAIN CANOA FRYNDIP (2,5KG)",882.5000,15060.80
4,850,"CONGELADO BATATA MCCAIN RUSTICA WEDGES (2,5KG)",870.0000,16234.51
5,908,"CONGELADO BATATA MCCAIN CRINKLE IMPORT (2,5KG)",750.0000,10278.82
6,909,CONGELADO BATATA MCCAIN MAXI CHIPS (2KG),14.0000,416.64
7,1000,"CONGELADO ANEL PRE FORMADO CEBOLA MCCAIN 1,05KG",530.2500,13041.67
8,1048,"CONGELADO BATATA MCCAIN CANOA NAC 6X2,5",535.0000,10632.80
9,1053,"CONGELADO BATATA MCCAIN HARVEST NAC 2,5KG",262.5000,4131.51
